In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import datatable as dt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
%time train = dt.fread("../input/jane-street-market-prediction/train.csv")
%time train = train.to_pandas()

In [ ]:
val = train[train['date'] > 400]
train = train[train['date'] <=400]

In [ ]:
features = [col for col in train.columns if 'feature' in col or col == 'date']

In [ ]:
x_train = train[features]
y_train = train['resp']
x_val = val[features]
y_val = val['resp']

In [ ]:
x_train = x_train.fillna(x_train.mean())
x_val = x_val.fillna(x_train.mean())

In [ ]:
y_train = y_train.fillna(y_train.mean())
y_val = y_val.fillna(y_train.mean())

In [ ]:
y_train = y_train>0
y_val = y_val>0

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 10, verbose = 2)

In [ ]:
%time rf.fit(x_train, y_train)

In [ ]:
feature_names = []
for i in range(130):
    feature_names.append('feature' + str(i))
feature_names.append('date')

In [ ]:
sorted_idx = rf.feature_importances_.argsort()

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize = (30,70))
ax.barh(y_ticks, rf.feature_importances_[sorted_idx])
ax.set_yticklabels(np.r_[feature_names][sorted_idx], fontsize = 18)
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

In [ ]:
y_predicted = rf.predict(x_val)

In [ ]:
accurate_preds = (y_predicted == y_val)

In [ ]:
sum(accurate_preds)/len(accurate_preds)

In [ ]:
from tqdm import tqdm
from math import sqrt
p = []
for date in tqdm(val['date'].unique()):
    p.append(val[val['date'] == date]['weight'].dot(val[val['date'] == date]['resp'].multiply(y_predicted[val[val['date'] == date]['resp'].index-val['resp'].index[0]])))
psq = [pi*pi for pi in p]
t = (sum(p)/sqrt(sum(psq)))*sqrt(250/(len(val['date'].unique())))
u = min(max(t,0),6)*sum(p)

In [ ]:
print("Utility score =", u, " za ", len(y_val), " tradeova")